# Artificial Intelligence Semester Project
## Name: Hammad Qaiser
## Roll no: 21I-2463

In [1]:
import random
from tabulate import tabulate
import matplotlib.pyplot as plt

# Define the classes for the timetable entities
class Course:
    def __init__(self, name, is_lab, instructors, max_students):
        self.name = name
        self.is_lab = is_lab
        self.instructors = instructors
        self.max_students = max_students

class Section:
    def __init__(self, name, strength):
        self.name = name
        self.strength = strength

class Professor:
    def __init__(self, name, max_courses):
        self.name = name
        self.max_courses = max_courses
        self.current_courses = 0

class Room:
    def __init__(self, name, capacity, room_type):
        self.name = name
        self.capacity = capacity
        self.room_type = room_type



# Constants
MAX_DAYS = 5
MAX_TIMESLOTS = 16  # 8 timeslots from 8:30 am to 5:30 pm
TIME_SLOT_DURATION = 80  # Duration of each slot in minutes

# Sample data
courses = [
    Course("SE", False, ["Dr. Alan", "Dr. Dar"], 60),
    Course("AI", False, ["Dr. Hammad", "Dr. Gul"], 50),
    Course("Algo", False, ["Dr. Newton", "Dr. kazmi"], 40),
    Course("OS", True, ["Dr. Ahmad", "Dr. Ali"], 30),
    Course("DB", False, ["Dr. Smith", "Dr. Gul"], 70)
]
sections = [Section("A", 50), Section("B", 40)]
professors = [Professor("Dr. Smith", 3), Professor("Dr. Johnson", 3), Professor("Dr. Moeed", 3), Professor("Dr. Gul", 3)]
rooms = [Room("Room1", 60, "classroom"), Room("Room2", 120, "large hall")]

# Function to generate a random timetable
def generate_random_timetable(courses, sections, professors, rooms):
    timetable = []
    for course in courses:
        for section in sections:
            if section.strength <= course.max_students:
                professor_choices = [prof for prof in professors if prof.name in course.instructors and prof.current_courses < prof.max_courses]
                if not professor_choices:
                    continue
                professor = random.choice(professor_choices)
                professor.current_courses += 1

                room_choices = [room for room in rooms if room.capacity >= section.strength]
                if not room_choices:
                    continue
                room = random.choice(room_choices)

                day1 = random.randint(1, MAX_DAYS - 1)
                day2 = random.randint(day1 + 1, MAX_DAYS)
                timeslot1 = random.randint(1, MAX_TIMESLOTS - 3)
                timeslot2 = random.randint(1, MAX_TIMESLOTS - 3)

                timeslot1_str = f"{timeslot1 * TIME_SLOT_DURATION // 60}:00-{(timeslot1 + 1) * TIME_SLOT_DURATION // 60}:20"
                timeslot2_str = f"{timeslot2 * TIME_SLOT_DURATION // 60}:00-{(timeslot2 + 1) * TIME_SLOT_DURATION // 60}:20"

                timetable.append((course.name, section.name, professor.name, room.name, day1, timeslot1_str))
                timetable.append((course.name, section.name, professor.name, room.name, day2, timeslot2_str))
    return timetable






In [2]:
# Function to calculate the fitness of a timetable
def calculate_fitness(timetable):
    hard_constraint_conflicts = 0
    soft_constraint_conflicts = 0

    professor_assignments = {}
    section_assignments = {}
    room_assignments = {}

    for entry in timetable:
        course_name, section_name, professor_name, room_name, day, timeslot = entry

        # Check for professor conflicts
        if professor_name in professor_assignments.get((day, timeslot), []):
            hard_constraint_conflicts += 1
        else:
            professor_assignments.setdefault((day, timeslot), []).append(professor_name)

        # Check for section conflicts
        if section_name in section_assignments.get((day, timeslot), []):
            hard_constraint_conflicts += 1
        else:
            section_assignments.setdefault((day, timeslot), []).append(section_name)

        # Check for room conflicts
        if room_name in room_assignments.get((day, timeslot), []):
            hard_constraint_conflicts += 1
        else:
            room_assignments.setdefault((day, timeslot), []).append(room_name)

        # Ensure theory in the morning and lab in the afternoon
        is_lab = "Lab" in course_name
        timeslot_hour = int(timeslot.split("-")[0].split(":")[0])
        if (is_lab and timeslot_hour < 14) or (not is_lab and timeslot_hour >= 14):
            soft_constraint_conflicts += 1

    fitness = -hard_constraint_conflicts - soft_constraint_conflicts * 0.5  # Weighing soft conflicts less
    conflicts = hard_constraint_conflicts + soft_constraint_conflicts
    return fitness, conflicts




In [3]:
# Tournament selection function
def tournament_selection(population, tournament_size):
    selected_parents = []
    if len(population) < tournament_size:
        return selected_parents
    for _ in range(len(population)):
        tournament = random.sample(population, min(tournament_size, len(population)))
        winner = max(tournament, key=lambda x: calculate_fitness(x)[0])
        selected_parents.append(winner)
    return selected_parents

# One-point crossover function
def one_point_crossover(parent1, parent2):
    if not parent1 or not parent2:
        return parent1, parent2
    crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Mutation function
def mutation(timetable, mutation_rate):
    mutated_timetable = timetable[:]
    for i in range(len(mutated_timetable)):
        if random.random() < mutation_rate:
            course_name, section_name, professor_name, room_name, day, timeslot = mutated_timetable[i]
            mutated_timetable[i] = (
                course_name,
                random.choice(sections).name,
                random.choice(professors).name,
                random.choice(rooms).name,
                random.randint(1, MAX_DAYS),
                f"{random.randint(1, MAX_TIMESLOTS - 3) * TIME_SLOT_DURATION // 60}:00-{random.randint(1, MAX_TIMESLOTS) * TIME_SLOT_DURATION // 60}:20"
            )
    return mutated_timetable

# Replace old population with the new population
def replace_population(old_population, new_population, population_size):
    combined_population = old_population + new_population
    combined_population.sort(key=lambda x: calculate_fitness(x)[0], reverse=True)
    return combined_population[:population_size]

In [4]:
def genetic_algorithm(courses, sections, professors, rooms, max_iterations, population_size, tournament_size, mutation_rate):
    population = [generate_random_timetable(courses, sections, professors, rooms) for _ in range(population_size)]

    for iteration in range(max_iterations):
        selected_parents = tournament_selection(population, tournament_size)
        offspring = []
        for i in range(0, len(selected_parents), 2):
            parent1 = selected_parents[i]
            if i+1 < len(selected_parents):
                parent2 = selected_parents[i+1]
                child1, child2 = one_point_crossover(parent1, parent2)
                offspring.extend([child1, child2])
            else:
                child = mutation(parent1, mutation_rate)
                offspring.append(child)

        mutated_offspring = [mutation(timetable, mutation_rate) for timetable in offspring]
        population = replace_population(population, mutated_offspring, population_size)

    return population


In [5]:


def plotTimetable(timetable):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    timeslots = [f"{(i+8):02}:30" for i in range(8, 18)] 

    plt.figure(figsize=(12, 8))

    for entry in timetable:
        course_name, section_name, professor_name, room_name, day, timeslot = entry
        day_index = day - 1
        timeslot_index = timeslots.index(timeslot.split("-")[0].split(":")[0] + ":30")
        plt.plot(day_index, timeslot_index, marker='o', color='blue')  # Plotting each class

        
        plt.annotate(f"{course_name}\n{professor_name}", (day_index, timeslot_index), textcoords="offset points", xytext=(-10,10), ha='center')

    plt.xticks(range(5), days)  # Labeling x-axis with days
    plt.yticks(range(len(timeslots)), timeslots)  # Labeling y-axis with time slots
    plt.title('Timetable Schedule')
    plt.xlabel('Day')
    plt.ylabel('Time')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [13]:
# Hammad Qaiser 21I-2463

def main():
    # Genetic Algorithm parameters
    max_iterations = 50
    population_size = 50
    tournament_size = 5
    mutation_rate = 0.05

    # Run the genetic algorithm
    print("\n" + "*"*60)
    print("*" + " "*58 + "*")
    print("*" + " "*20 + "Welcome to the Class Schedule Generator" + " "*20 + "*")
    print("*" + " "*58 + "*")
    print("*"*60 + "\n")

    print("Generating the schedule...\n")

    final_population = genetic_algorithm(courses, sections, professors, rooms, max_iterations, population_size, tournament_size, mutation_rate)

    print("\n" + "*"*60)
    print("*" + " "*58 + "*")
    print("*" + " "*23 + "End of Schedule" + " "*23 + "*")
    print("*" + " "*58 + "*")
    print("*"*60 + "\n")

    # Print the final timetables
    for i, timetable in enumerate(final_population):
        fitness, conflicts = calculate_fitness(timetable)
        print(f"Fitness of Timetable {i+1}: {fitness}, Conflicts: {conflicts}")
        print("Timetable:")
        headers = ["Index", "Course Name", "Section", "Professor", "Room", "Day", "Time", "Room Capacity"]
        data = []
        for idx, entry in enumerate(timetable):
            room_capacity = next((room.capacity for room in rooms if room.name == entry[3]), "Unknown")
            data.append([idx+1, entry[0], entry[1], entry[2], entry[3], f"Day {entry[4]}", entry[5], room_capacity])
        print(tabulate(data, headers=headers, tablefmt="grid"))
        print()

if __name__ == "__main__":
    main()



************************************************************
*                                                          *
*                    Welcome to the Class Schedule Generator                    *
*                                                          *
************************************************************

Generating the schedule...


************************************************************
*                                                          *
*                       End of Schedule                       *
*                                                          *
************************************************************

Fitness of Timetable 1: 0.0, Conflicts: 0
Timetable:
+---------+---------------+-----------+-------------+--------+-------+--------+-----------------+
| Index   | Course Name   | Section   | Professor   | Room   | Day   | Time   | Room Capacity   |
+=========+===============+===========+=============+========+=======+========+======